In [1]:
import numpy as np

# Plan działania
Spróbujemy znaleźć rozkład:
$$PAP^T = U\Sigma V^T$$

Wówczas rozkład SVD macierzy $A$ odzyskujemy jako:
$$A = \tilde{U}\Sigma \tilde{V}^T$$
gdzie $\tilde{U} = P^TU, \tilde{V}^T = PV^T$. Celem jest połączenie efektów uzyskanych w plikach "pierwszemysli" oraz "drugiemysli".

Uwaga: na razie w ogóle nie zajmujemy się macierzami $U$ oraz $V$ - szukamy tylko odpowiednich wartości własnych.

In [79]:
# Teraz trzeba rozwiązać pełne zadanie własne macierzy D^2 + z*z^T
# W tym celu stworzymy sobie inną macierz diagonalną D oraz inny wektor z
D = np.diag([-0.11, 0.23, 0.23, 0.23, 0.38, 0.49, 1.13])
z = np.array([0.13, -2.11, 0, 3.37, 5.18, 0.92, 0])

ind = sum(z == 0)
perm = (z == 0)

N = D.shape[0]

A = D + z.reshape((N, 1)) @ z.reshape((1, N))

In [80]:
# Permutujemy zgodnie z notatkami
PAPt = np.block([
    [A[perm, :][:, perm], A[perm, :][:, ~perm]],
    [A[~perm, :][:, perm], A[~perm, :][:, ~perm]]
]) # przepermutowane A, czyli macierz P*A*P'

PDPt = np.diag(np.hstack((np.diag(D)[perm], np.diag(D)[~perm])))

Ahat = A[~perm, :][:, ~perm] # to, co zostało do rozłożenia z macierzy "A" (prawy dolny róg PAPt)
Dhat = D[~perm, :][:, ~perm] # odpowiadająca pozostałość macierzy "D"
zhat = z[~perm] # niezerowe elementy "z"

Pz = np.hstack((z[perm], z[~perm])) # przepermutowany "z"

Szybkie sprawdzenie: powinno być $\hat{A} = \hat{D} + \hat{z}\hat{z}^T$.

In [81]:
np.all(np.isclose(Ahat, Dhat + zhat.reshape((N-ind, 1))@zhat.reshape((1, N-ind))))

True

Wszystko działa, zatem idziemy dalej.

In [82]:
def count_elements(arr: np.array) -> dict:
    
    retdict = {}
    for i, a in enumerate(arr):
        if retdict.get(a) is None:
            retdict[a] = [i]
        else:
            retdict[a].append(i)
    return retdict

ce = count_elements(np.diag(Dhat))

In [83]:
Nhat = Ahat.shape[0]

In [84]:
for key, val in ce.items():
    if len(val) > 1:
        reps = True
        print(f"{key} jest {len(val)-1}-krotną wartością własną Ahat.")
        i = val[0]
        k = len(val)
        temp_zhat = zhat[val]
        
        u = np.copy(temp_zhat)
        u[-1] += np.linalg.norm(u)
        v = u/np.linalg.norm(u)
        Hi = np.eye(k) - 2 * v.reshape((k, 1)) @ v.reshape((1, k))
        
        H = np.block([
                        [np.eye(i),               np.zeros((i, k)),         np.zeros((i, Nhat-i-k))],
                        [np.zeros((k, i)),        Hi,                       np.zeros((k, Nhat-i-k))],
                        [np.zeros((Nhat-i-k, i)), np.zeros((Nhat-i-k, k)),  np.eye(Nhat-i-k)]
                        ])
        break
if not reps:
    print(f"Macierz Dhat nie zawiera powtórzeń na diagonali.")

0.23 jest 1-krotną wartością własną Ahat.


In [85]:
Ahat

array([[ -0.0931,  -0.2743,   0.4381,   0.6734,   0.1196],
       [ -0.2743,   4.6821,  -7.1107, -10.9298,  -1.9412],
       [  0.4381,  -7.1107,  11.5869,  17.4566,   3.1004],
       [  0.6734, -10.9298,  17.4566,  27.2124,   4.7656],
       [  0.1196,  -1.9412,   3.1004,   4.7656,   1.3364]])

In [86]:
ztilde = H @ zhat

In [87]:
np.all(np.isclose(Dhat + ztilde.reshape((Nhat, 1)) @ ztilde.reshape((1, Nhat)), H @ Ahat @ H.T))

True

In [88]:
H

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.84757415,  0.53067699,  0.        ,  0.        ],
       [ 0.        ,  0.53067699, -0.84757415,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ]])

Zdefiniujmy jeszcze blokową macierz $HH = \begin{bmatrix} I & 0 \\ 0 & H\end{bmatrix}$ i na niej sprawdźmy kilka działań.

Uwaga: poniżej cały czas działamy na macierzach spermutowanych!

In [89]:
HH = np.block([
    [np.eye(ind), np.zeros((ind, N-ind))],
    [np.zeros((N-ind, ind)), H]
    ])

In [94]:
ZTilde = HH @ Pz
ZTilde # działanie na przepermutowanym wektorze "z" jest ok

array([ 0.        ,  0.        ,  0.13      ,  0.        , -3.97605332,
        5.18      ,  0.92      ])

In [91]:
HH @ PAPt @ HH.T

array([[ 2.30000000e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  1.13000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -9.31000000e-02,
         1.92643728e-17, -5.16886932e-01,  6.73400000e-01,
         1.19600000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.92643728e-17,
         2.30000000e-01, -6.47803115e-17,  5.71443945e-17,
        -1.28256410e-16],
       [ 0.00000000e+00,  0.00000000e+00, -5.16886932e-01,
        -1.10899829e-15,  1.60390000e+01, -2.05959562e+01,
        -3.65796905e+00],
       [ 0.00000000e+00,  0.00000000e+00,  6.73400000e-01,
         5.71443945e-17, -2.05959562e+01,  2.72124000e+01,
         4.76560000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.19600000e-01,
        -1.28256410e-16, -3.65796905e+00,  4.76560000e+00,
         1.3364000

In [95]:
np.all(np.isclose(PDPt + ZTilde.reshape((N, 1)) @ ZTilde.reshape((1, N)), HH @ PAPt @ HH.T))

True

Teraz: zgodnie z notatkami, powinno się łatwo dać odzyskać wartości własne i wektory własne macierzy $H\hat{A} H^T$.